In [ ]:
import ray
import ray.rllib.algorithms.cql as cql
from ray.tune.logger import pretty_print

ray.init(ignore_reinit_error=True)

# 复制默认配置，并指定离线数据路径
config = cql.DEFAULT_CONFIG.copy()
config["num_workers"] = 1
config["framework"] = "torch"
# 指定离线数据输入路径，RLlib 会自动加载该目录下的 JSON Lines 文件
config["input"] = "tmp/pendulum-out"

# 注意：离线训练时，算法不会与环境交互采集数据
algo = cql.CQL(config=config, env="Pendulum-v1")

# 离线训练循环
for i in range(100):
    result = algo.train()
    print(pretty_print(result))

    if i % 10 == 0:
        checkpoint = algo.save("save_model")
        print("checkpoint saved at", checkpoint)

ray.shutdown()


2025-03-17 22:49:50,990	INFO worker.py:1351 -- Calling ray.init() again after it has already been called.
2025-03-17 22:49:50,994	WARNING deprecation.py:47 -- DeprecationWarning: `config['multiagent']['replay_mode']` has been deprecated. config['replay_buffer_config']['replay_mode'] This will raise an error in the future!
2025-03-17 22:49:52,365	WARNING json_reader.py:166 -- Treating input directory as glob patterns: ['d:\\Desktop\\CQL\\path\\to\\offline_data\\*.json', 'd:\\Desktop\\CQL\\path\\to\\offline_data\\*.zip']
2025-03-17 22:49:52,367	WARNING util.py:65 -- Install gputil for GPU system monitoring.
2025-03-17 22:49:52,392	WARNING algorithm.py:2178 -- Worker crashed during training or evaluation! To try to continue without failed worker(s), set `ignore_worker_failures=True`. To try to recover the failed worker(s), set `recreate_failed_workers=True`.


RayTaskError(ValueError): [36mray::RolloutWorker.sample()[39m (pid=58604, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000016308F04B20>)
  File "python\ray\_raylet.pyx", line 662, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 666, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 613, in ray._raylet.execute_task.function_executor
  File "d:\miniconda3\envs\torch191\lib\site-packages\ray\_private\function_manager.py", line 674, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "d:\miniconda3\envs\torch191\lib\site-packages\ray\util\tracing\tracing_helper.py", line 466, in _resume_span
    return method(self, *_args, **_kwargs)
  File "d:\miniconda3\envs\torch191\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 806, in sample
    batches = [self.input_reader.next()]
  File "d:\miniconda3\envs\torch191\lib\site-packages\ray\rllib\offline\shuffled_input.py", line 34, in 

In [ ]:
import gym
import ray
import ray.rllib.algorithms.cql as cql

ray.init(ignore_reinit_error=True)
config = cql.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["framework"] = "torch"
algo = cql.CQL(config=config, env="Pendulum-v1")

# 加载之前保存的checkpoint（请将此路径替换为实际的checkpoint路径）
checkpoint_path = "save_model/your_checkpoint_directory_or_file"  
algo.restore(checkpoint_path)

# 使用加载的模型在环境中进行测试
env = gym.make("Pendulum-v1")
obs = env.reset()
done = False
while not done:
    action = algo.compute_action(obs)
    obs, reward, done, info = env.step(action)
    env.render()

ray.shutdown()
